# Тестовое задание

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import string

In [ ]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
# tok = tfidf_vect.build_tokenizer()
# tok = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize
clf = LogisticRegression(solver='liblinear', class_weight = {0:4, 1:1}, penalty='l2')

## Cкачиваем файл с данными

In [ ]:
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [ ]:
df = pd.read_json ('Electronics_5.json', lines=True )
df.to_csv ('Electronics_5.csv', index = None)

ValueError: ignored

# Предобработка

In [ ]:
dat = pd.read_csv("Electronics_5.csv", engine= 'python', error_bad_lines=False)[["asin","reviewText", "overall"]]

Skipping line 160645: unexpected end of data


## Добавляем бинрную оценку

In [ ]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [ ]:
dat_t, dat = dat, dat[:16000]

In [ ]:
train_dat, privat_dat, train_ans, privat_ans = train_test_split(
    dat.reviewText, dat.bin, test_size=0.15)

In [ ]:
train_dat, test_dat, train_ans, test_ans = train_test_split(
    train_dat, train_ans, test_size=0.2)

## Натравим на необработанные данные

In [ ]:
count_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
tfidf_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
count_train_mtrx_dft = count_vect.transform(train_dat.apply(lambda x: np.str_(x)))
tfidf_train_mtrx_dft = tfidf_vect.transform(train_dat.apply(lambda x: np.str_(x)))
count_test_mtrx_dft = count_vect.transform(test_dat.apply(lambda x: np.str_(x)))
tfidf_test_mtrx_dft = tfidf_vect.transform(test_dat.apply(lambda x: np.str_(x)))

## Приводим к базовой форме

In [ ]:
train_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), train_dat.apply(lambda x: np.str_(x)))))
test_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), test_dat.apply(lambda x: np.str_(x)))))
privat_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), privat_dat.apply(lambda x: np.str_(x)))))
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Натравим на обработанные данные

In [ ]:
# count_vect.fit(dat.words)
# tfidf_vect.fit(dat.words)
count_train_mtrx = count_vect.transform(train_dat_tr)
tfidf_train_mtrx = tfidf_vect.transform(train_dat_tr)
count_test_mtrx = count_vect.transform(test_dat_tr)
tfidf_test_mtrx = tfidf_vect.transform(test_dat_tr)

## Проверим, что получилось

In [ ]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
15995,B00004ZCJE,I could not ask more of the Tiffen 58mm UV Pro...,5,True,could not ask more of the tiffen 58mm uv prote...
15996,B00004ZCJE,I ordered 2 Tiffen 52 mm UV filters for the 2 ...,4,True,order tiffen 52 mm uv filter for the lens that...
15997,B00004ZCJE,It protects the lens. But I still have glare f...,4,True,it protect the len but still have glare from l...
15998,B00004ZCJE,"buena calidad, resistente, producto bien hecho...",5,True,buena calidad resistent producto bien hecho la...


# Обучим модели и посмотрим их точность

## Обработанные с оптимальными параметрами

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx, train_ans)
print(search.best_params_)

{'C': 0.1, 'class_weight': None, 'penalty': 'l2'}


In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx))

0.8647058823529412

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx,  train_ans)
print(search.best_params_)

{'C': 2.7, 'class_weight': None, 'penalty': 'l1'}


In [ ]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx))

0.8713235294117647

## Дефолтные с оптимальными параметрами

In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}, {0:1, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 0.2, 'class_weight': None, 'penalty': 'l2'}


In [ ]:
print(accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx_dft)))
print(classification_report(test_ans, search.best_estimator_.predict(count_test_mtrx_dft)))

0.8702205882352941
              precision    recall  f1-score   support

       False       0.71      0.48      0.58       496
        True       0.89      0.96      0.92      2224

    accuracy                           0.87      2720
   macro avg       0.80      0.72      0.75      2720
weighted avg       0.86      0.87      0.86      2720



In [ ]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None, {0:4, 1:1}]
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 2.6, 'class_weight': {0: 4, 1: 1}, 'penalty': 'l2'}


In [ ]:
print(accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft)))
print(classification_report(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft)))


0.8735294117647059
              precision    recall  f1-score   support

       False       0.65      0.67      0.66       496
        True       0.93      0.92      0.92      2224

    accuracy                           0.87      2720
   macro avg       0.79      0.79      0.79      2720
weighted avg       0.88      0.87      0.87      2720



# Лучший результат у tfidf на необработанных данных

In [ ]:
clf = LogisticRegression(solver='liblinear', class_weight = {0:4, 1:1}, C=2.6, penalty='l2')
clf.fit(tfidf_train_mtrx_dft, train_ans)
pred = clf.predict(tfidf_vect.transform(privat_dat.apply(lambda x: np.str_(x))))
print(accuracy_score(pred, privat_ans))
print(classification_report(pred, privat_ans))

0.8658333333333333
              precision    recall  f1-score   support

       False       0.60      0.59      0.59       400
        True       0.92      0.92      0.92      2000

    accuracy                           0.87      2400
   macro avg       0.76      0.76      0.76      2400
weighted avg       0.87      0.87      0.87      2400



## Проверим баланс классов

In [ ]:
print("All :\n1 - ", sum(dat.bin), "\n0 - ", len(dat.bin) - sum(dat.bin), "\n", round(100*sum(dat.bin)/len(dat.bin), 2)) 
print("\nTrain :\n1 - ", sum(train_ans), "\n0 - ", len(train_ans) - sum(train_ans), "\n", round(100*sum(train_ans)/len(train_ans), 2)) 
print("\nTest :\n1 - ", sum(test_ans), "\n0 - ", len(test_ans) - sum(test_ans), "\n", round(100*sum(test_ans)/len(test_ans), 2) )
print("\nPrivat_ans :\n1 - ", sum(privat_ans), "\n0 - ", len(privat_ans) - sum(privat_ans), "\n", round(100*sum(privat_ans)/len(privat_ans), 2) ) 

All :
1 -  13194 
0 -  2806 
 82.46

Train :
1 -  8964 
0 -  1916 
 82.39

Test :
1 -  2224 
0 -  496 
 81.76

Privat_ans :
1 -  2006 
0 -  394 
 83.58


# Возьмем новый набор данных

In [ ]:
dat2 = pd.read_json("Tools_and_Home_Improvement_5.json", lines = True)[["reviewText", "overall"]]

In [ ]:
dat2["bin"] = dat2.overall > 3

## Истинное значение

In [ ]:
rat = (len(dat2.bin)-sum(dat2.bin)) / sum(dat2.bin)
print("Соотношение false/true:",len(dat2.bin)-sum(dat2.bin), " / ", sum(dat2.bin))
print(accuracy_score(dat2.bin, search.best_estimator_.predict(tfidf_vect.transform(dat2.reviewText))))
print(classification_report(dat2.bin, search.best_estimator_.predict(tfidf_vect.transform(dat2.reviewText))))

Соотношение false/true: 20874  /  113602
0.84983194027187
              precision    recall  f1-score   support

       False       0.51      0.64      0.57     20874
        True       0.93      0.89      0.91    113602

    accuracy                           0.85    134476
   macro avg       0.72      0.76      0.74    134476
weighted avg       0.87      0.85      0.86    134476



## Случайный выбор

In [ ]:
df_rand_ch = dat2.sample(4000)
print("Соотношение false/true:",len(df_rand_ch.bin)-sum(df_rand_ch.bin), " / ", sum(df_rand_ch.bin))
print(accuracy_score(df_rand_ch.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rand_ch.reviewText))))
print(classification_report(df_rand_ch.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rand_ch.reviewText))))

Соотношение false/true: 639  /  3361
0.8515
              precision    recall  f1-score   support

       False       0.53      0.65      0.58       639
        True       0.93      0.89      0.91      3361

    accuracy                           0.85      4000
   macro avg       0.73      0.77      0.75      4000
weighted avg       0.87      0.85      0.86      4000



## Случайный равномерный выбор

In [ ]:
dat2["Choice"] = 0
for i in range(len(dat2.bin)):
    dat2['Choice'][i] = (np.random.randint(round(10 / rat)) if dat2.bin[i] else np.random.randint(round(10)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
new_test_dat = dat2.loc[dat2.Choice == 0][["reviewText", "overall", "bin", "Choice"]]
new_test_dat

,reviewText,overall,bin,Choice
17,"From reading all the glowing reviews here, I t...",3,False,0
26,This light is useless for reading in bed next ...,2,False,0
46,Good price and works well. I use it in my Ken...,5,True,0
60,"this is not a standard size bulb, it is smalle...",3,False,0
70,"The bulbs came all in one small ziplock bag, w...",3,False,0
...,...,...,...,...
134218,"Yes, this is easy to retrofit; the easy video ...",3,False,0
134223,This is not what I thought it was when I order...,3,False,0
134291,"Really like this light, its does the job and m...",5,True,0
134341,"Bought this so I can use thus with my nook, I ...",5,True,0


In [ ]:
print("Соотношение false/true:",len(new_test_dat.bin)-sum(new_test_dat.bin), " / ", sum(new_test_dat.bin))
print("Всего " , (len(new_test_dat.bin)))
print(accuracy_score(new_test_dat.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat.reviewText))))
print(classification_report(new_test_dat.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat.reviewText))))

Соотношение false/true: 2132  /  2086
Всего  4218
0.7726410621147464
              precision    recall  f1-score   support

       False       0.86      0.66      0.75      2132
        True       0.72      0.89      0.79      2086

    accuracy                           0.77      4218
   macro avg       0.79      0.77      0.77      4218
weighted avg       0.79      0.77      0.77      4218



## Случайный выбор с сохраненем баланса классов

In [ ]:
df_rc_2 = pd.concat([dat2.loc[dat2['bin']].sample(round(4000*(1/(rat+1)))), dat2.loc[dat2['bin'] != True].sample(round(4000*rat/(rat+1)))], ignore_index=True)
print("Соотношение false/true:",len(df_rc_2.bin)-sum(df_rc_2.bin), " / ", sum(df_rc_2.bin))
print(accuracy_score(df_rc_2.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rc_2.reviewText))))
print(classification_report(df_rc_2.bin, search.best_estimator_.predict(tfidf_vect.transform(df_rc_2.reviewText))))

Соотношение false/true: 621  /  3379
0.84025
              precision    recall  f1-score   support

       False       0.49      0.60      0.54       621
        True       0.92      0.88      0.90      3379

    accuracy                           0.84      4000
   macro avg       0.71      0.74      0.72      4000
weighted avg       0.86      0.84      0.85      4000



## По уверености классификатора

In [ ]:
prob = search.best_estimator_.predict_proba(tfidf_vect.transform(dat2.reviewText))
prob = [x[0] for x in prob]

In [ ]:
dat2["chance"] = prob

### Уверенность, что оценка <= 3

In [ ]:
dat2 = dat2.sort_values(by = "chance")

In [ ]:
dat2

,reviewText,overall,bin,Choice,chance
43376,Great thermostat,4,True,26,0.000020
50418,GREAT,5,True,35,0.000020
51595,Great hammer.,5,True,21,0.000020
27435,GREAT.. ........... .......... ............ .....,5,True,28,0.000020
52596,great sharpener,5,True,2,0.000020
...,...,...,...,...,...
98628,"At first, we were very pleased with the STI-34...",1,False,9,0.996168
92229,The metal of a product is not measured by how ...,2,False,4,0.996580
31175,This seemed OK when I first got it. But it sto...,1,False,9,0.997447
116804,what a waste of my time. All 5 I ordered did ...,1,False,0,0.997805


In [ ]:
for i in range(len(dat2.Choice)):
    dat2.Choice[i] = (i % 32 == 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
new_test_dat_2 = dat2.loc[dat2.Choice]
new_test_dat_2

,reviewText,overall,bin,Choice,chance
61600,Works great.,5,True,True,0.000289
37344,Great jig saw. The light is great and makes i...,5,True,True,0.001163
130208,This is a great buy. Love the color and price.,5,True,True,0.001590
128096,This is one of the best flashlights I have com...,5,True,True,0.001881
70656,I love this system. I've used it to build sev...,5,True,True,0.002094
...,...,...,...,...,...
91904,We bought 4 of these back in 2010. One has st...,2,False,True,0.988569
60384,I have had this replaced 3 times now thru the ...,1,False,True,0.989966
121568,The flashlight casing is ok but the charger un...,1,False,True,0.990595
57056,Very expensive and just equally as worthless. ...,1,False,True,0.993496


In [ ]:
print("Соотношение false/true:",len(new_test_dat_2.bin)-sum(new_test_dat_2.bin), " / ", sum(new_test_dat_2.bin))
print("Всего " , (len(new_test_dat_2.bin)))

Соотношение false/true: 639  /  3564
Всего  4203


In [ ]:
print(accuracy_score(new_test_dat_2.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat_2.reviewText))))
print(classification_report(new_test_dat_2.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat_2.reviewText))))

0.8532000951701166
              precision    recall  f1-score   support

       False       0.51      0.64      0.57       639
        True       0.93      0.89      0.91      3564

    accuracy                           0.85      4203
   macro avg       0.72      0.76      0.74      4203
weighted avg       0.87      0.85      0.86      4203

